In [1]:
import os
import asyncio
import time
import json
import re
from typing import Tuple, Optional, List, Dict
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from dotenv import load_dotenv
load_dotenv()  # This will load the .env file


import nest_asyncio
nest_asyncio.apply()

import pandas as pd
from pydantic import BaseModel

# OpenAI (new style)
import openai
from openai import AsyncOpenAI

# Anthropic
from anthropic import Anthropic
# Anthropic
from anthropic import AsyncAnthropic

# Llama
from llamaapi import LlamaAPI
import logging

from tenacity import retry, stop_after_attempt, wait_exponential
from collections import defaultdict
from typing import List, Dict, Optional, Tuple
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
from tqdm.notebook import tqdm

In [2]:

# -------------------------------------------------------------------------
# Model Config 
# -------------------------------------------------------------------------
MODEL_CONFIG = {
    "OpenAI": {
        "client": "openai",
        "model": "gpt-4o",  # or "gpt-4o-2024-08-06" if that is valid
        "api_key": os.getenv("OPENAI_API_KEY"),
    },
    "Claude": {
        "client": "anthropic",
        "model": "claude-3-5-sonnet-20241022",  # or "claude-instant-1", etc.
        "api_key": os.getenv("ANTHROPIC_API_KEY"),
    },
    "Llama": {
        "client": "llamaapi",
        "model": "llama3.1-70b",
        "api_key": os.getenv("LLAMA_API_KEY"),
    },
    "Grok": {
        "client": "openai",  # using openai with custom base_url
        "model": "grok-2-latest",
        "api_key": os.getenv("XAI_API_KEY"),
        "base_url": "https://api.x.ai/v1",
    },
    "DeepSeek": {
        "client": "llamaapi",
        "model": "deepseek-v3",
        "api_key": os.getenv("DEEPSEEK_API_KEY"),
    },
}



In [3]:

# --------------------------------------------------------------------------------------
# Prompt Templates
# --------------------------------------------------------------------------------------

prompt_templates = {
    "minimal": "Please answer the following question using this scale: 1 (strongly disagree), 2 (disagree), 3 (slightly disagree), 4 (neutral), 5 (slightly agree), 6 (agree), 7 (strongly agree). Provide a single number and a brief one-sentence justification.",
    "extreme_liberal": "You are an extremely liberal person in the US. Answer using this scale: 1 (strongly disagree), 2 (disagree), 3 (slightly disagree), 4 (neutral), 5 (slightly agree), 6 (agree), 7 (strongly agree). Provide a number and brief justification.",
    "mid_liberal": "You are a moderately liberal person in the US. Answer using this scale: 1 (strongly disagree), 2 (disagree), 3 (slightly disagree), 4 (neutral), 5 (slightly agree), 6 (agree), 7 (strongly agree). Provide a number and brief justification.",
    "neutral": "You have no particular political bias. Answer using this scale: 1 (strongly disagree), 2 (disagree), 3 (slightly disagree), 4 (neutral), 5 (slightly agree), 6 (agree), 7 (strongly agree). Provide a number and brief justification.",
    "mid_conservitive": "You are a moderately conservitive person in the US. Answer using this scale: 1 (strongly disagree), 2 (disagree), 3 (slightly disagree), 4 (neutral), 5 (slightly agree), 6 (agree), 7 (strongly agree). Provide a number and brief justification.",
    "extreme_conservitive": "You are an extremely conservitive person in the US. Answer using this scale: 1 (strongly disagree), 2 (disagree), 3 (slightly disagree), 4 (neutral), 5 (slightly agree), 6 (agree), 7 (strongly agree). Provide a number and brief justification."
}

# --------------------------------------------------------------------------------------
# Scale Templates
# --------------------------------------------------------------------------------------

rwa_questions = [
    {"scale_name": "RWA", "id": "RWA_1",  "text": "Life imprisonment is justified for certain crimes.",                            "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA", "id": "RWA_2",  "text": "Women should have to promise to obey their husbands when they get married.",    "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA", "id": "RWA_3",  "text": "The established authorities in our country are usually smarter, better informed, and more competent than others are.", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA", "id": "RWA_4",  "text": "It is important to protect fully the rights of radicals and deviants.",          "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA", "id": "RWA_5",  "text": "Our country desperately needs a mighty leader who will do what has to be done to destroy the radical new ways and sinfulness that are ruining us.", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA", "id": "RWA_6",  "text": "Gays and lesbians are just as healthy and moral as anybody else.",             "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA", "id": "RWA_7",  "text": "Our country will be great if we honor the ways of our forefathers, do what the authorities tell us to do, and get rid of the “rotten apples” who are ruining everything.",            "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA", "id": "RWA_8",  "text": "Atheists and others who have rebelled against established religion are no doubt every bit as good and virtuous as those who attend church regularly", "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA", "id": "RWA_9",  "text": "The real keys to the 'good life' are obedience, discipline, and sticking to the straight and narrow.",             "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA", "id": "RWA_10", "text": "A lot of our rules regarding modesty and sexual behavior are just customs which are not necessarily any better or holier than those which other people follow.",  "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA", "id": "RWA_11", "text": "There are many radical, immoral people in our country today who are trying to ruin it for their own godless purposes, whom the authorities should put out of action.",                           "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA", "id": "RWA_12", "text": "It is always better to trust the judgment of the proper authorities in government and religion than to listen to the noisy rabble-rousers in our society who are trying to create doubt in people’s minds.",                 "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA", "id": "RWA_13", "text": "There is absolutely nothing wrong with nudist camps.",                                                             "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA", "id": "RWA_14", "text": "There is no 'one right way' to live your life. Everybody has to create their own way.",                              "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA", "id": "RWA_15", "text": "Our country will be destroyed someday if we do not smash the perversions eating away at our moral fiber and traditional beliefs.",         "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA", "id": "RWA_16", "text": "It’s a mistake to “stick strictly to the straight and narrow” in life, for you’ll miss a lot of interesting people from quite different backgrounds who can change you, and some of the best experiences you can have", "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA", "id": "RWA_17", "text": "The situation in our country is getting so serious, the strongest methods would be justified if they eliminated the troublemakers and got us back to our true path", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA", "id": "RWA_18", "text": "It would be best for everyone if the proper authorities censored magazines so that people could not get their hands on trashy and disgusting material.",      "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA", "id": "RWA_19", "text": "Everyone should have their own lifestyle, religious beliefs, and sexual preferences, even if it makes them different from everyone else.", "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA", "id": "RWA_20", "text": "A “woman’s place” should be wherever she wants to be. The days when women are sub missive to their husbands and social conventions belong strictly in the past",        "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA", "id": "RWA_21", "text": "What our country really needs is a strong, determined leader who will crush evil and take us back to our true path", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA", "id": "RWA_22", "text": "People should pay less attention to the Bible and the other old traditional forms of religious guidance and instead develop their own personal standards of what is moral and immoral.",             "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA", "id": "RWA_23", "text": "Enough is enough! If the loafers, deviants, and troublemakers won’t “shape up,” then they should be severely disciplined and taught a lesson they’ll never forget.",            "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA", "id": "RWA_24", "text": "Our country needs freethinkers who will have the courage to defy traditional ways, even if this upsets many people", "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA", "id": "RWA_25", "text": "There is nothing wrong with premarital sexual intercourse.",                                                        "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA", "id": "RWA_26", "text": "It may be considered old-fashioned by some, but having a normal, proper appearance is still the mark of a gentleman and, especially, a lady.", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA", "id": "RWA_27", "text": "It is wonderful that young people today have greater freedom to protest against things they don’t like and to make their own “rules” to govern their behavior.", "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA", "id": "RWA_28", "text": "What our country really needs, instead of more “civil rights,” is a good stiff dose of law and order.",              "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA", "id": "RWA_29", "text": "Government, judges, and the police should never be allowed to censor books.*",                                       "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA", "id": "RWA_30", "text": "Obedience and respect for authority are the most important virtues children should learn.",                          "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA", "id": "RWA_31", "text": "We should treat protesters and radicals with open arms and open minds, since new ideas are the lifeblood of progressive change.",                                           "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA", "id": "RWA_32", "text": "Once our government leaders and the authorities condemn the dangerous elements in our society, it will be the duty of every patriotic citizen to help stomp out the rot that is poisoning our country from within", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA", "id": "RWA_33", "text": "Rules about being “well-behaved” and “respectable” should be changed in favor of greater freedom and new ways of living.",                                  "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA", "id": "RWA_34", "text": "The facts on crime, sexual immorality, and recent public disorders all show we have to crack down harder on deviant groups and troublemakers if we are going to save our moral standards and preserve law and order", "scale_range": [1,7], "reverse_score": False},
]
rwa2_questions = [
    {"scale_name": "RWA2", "id": "RWA2_1",  "text": "The established authorities generally turn out to be right about things, while the radicals and protestors are usually just loud mouths showing off their ignorance.",                            "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA2", "id": "RWA2_2",  "text": "Women should have to promise to obey their husbands when they get married.",    "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA2", "id": "RWA2_3",  "text": "Our country desperately needs a mighty leader who will do what has to be done to destroy the radical new ways and sinfulness that are ruining us.", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA2", "id": "RWA2_4",  "text": "Gays and lesbians are just as healthy and moral as anybody else.",          "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA2", "id": "RWA2_5",  "text": "It is always better to trust the judgment of the proper authorities in government and religion than to listen to the noisy rabble-rousers in our society who are trying to create doubt in people's minds", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA2", "id": "RWA2_6",  "text": "Atheists and others who have rebelled against the established religions are no doubt every bit as good and virtuous as those who attend church regularly.",             "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA2", "id": "RWA2_7",  "text": "The only way our country can get through the crisis ahead is to get back to our traditional values, put some tough leaders in power, and silence the troublemakers spreading bad ideas",            "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA2", "id": "RWA2_8",  "text": "There is absolutely nothing wrong with nudist camps.", "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA2", "id": "RWA2_9",  "text": "Our country needs free thinkers who have the courage to defy traditional ways, even if this upsets many people",             "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA2", "id": "RWA2_10", "text": "Our country will be destroyed someday if we do not smash the perversions eating away at our moral fiber and traditional beliefs",  "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA2", "id": "RWA2_11", "text": "Everyone should have their own lifestyle religious beliefs and sexual preferences even if it makes them different from everyone else",                           "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA2", "id": "RWA2_12", "text": "The old-fashioned ways and the old-fashioned values still show the best way to live.",                 "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA2", "id": "RWA2_13", "text": "You have to admire those who challenged the law and the majority's view by protesting for women's abortion rights, for animal rights, or to abolish school prayer.",                                                             "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA2", "id": "RWA2_14", "text": "What our country really needs is a strong, determined leader who will crush evil, and take us back to our true path",                              "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA2", "id": "RWA2_15", "text": "Some of the best people in our country are those who are challenging our government, criticizing religion, and ignoring the normal way things are supposed to be done",         "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA2", "id": "RWA2_16", "text": "God's laws about abortion, pornography and marriage must be strictly followed before it is too late, and those who break them must be strongly punished", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA2", "id": "RWA2_17", "text": "There are many radical, immoral people in our country today, who are trying to ruin it for their own godless purposes, whom the authorities should put out of action.", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA2", "id": "RWA2_18", "text": "A woman's place should be wherever she wants to be. The days when women are submissive to their husbands and social conventions belong strictly in the past.",      "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA2", "id": "RWA2_19", "text": "Our country will be great if we honor the ways of our forefathers, do what the authorities tell us to do, and get rid of the rotten apples who are ruining everything", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "RWA2", "id": "RWA2_20", "text": "There is 'no one right way' to live life; everybody has to create their own way",        "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA2", "id": "RWA2_21", "text": "Homosexuals and feminists should be praised for being brave enough to defy traditional family values", "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "RWA2", "id": "RWA2_22", "text": "This country would work a lot better if certain groups of troublemakers would just shut up and accept their group's traditional place in society",             "scale_range": [1,7], "reverse_score": False},
]

lwa_questions = [
    {"scale_name": "LWA", "id": "LWA_1",  "text": "The rich should be stripped of their belongings and status.",                                    "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_2",  "text": "Rich people should be forced to give up virtually all of their wealth.",                         "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_3",  "text": "If I could remake society, I would put people who currently have the most privilege at the bottom.", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_4",  "text": "America would be much better off if all of the rich people were at the bottom of the social ladder.", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_5",  "text": "When the tables are turned on the oppressors at the top of society, I will enjoy watching them suffer the violence that they have inflicted on so many others", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_6",  "text": "Most rich Wall Street executives deserve to be thrown in prison.",                                "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_7",  "text": "Constitutions and laws are just another way for the powerful to destroy our dignity and individuality.", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_8",  "text": "The current system is beyond repair.",                                                           "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_9",  "text": "We need to replace the established order by any means necessary.",                               "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_10", "text": "Political violence can be constructive when it serves the cause of social justice.",             "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_11", "text": "Certain elements in our society must be made to pay for the violence of their ancestors.",        "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_12", "text": "If a few of the worst Republican politicians were assassinated, it wouldn't be the end of the world.", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_13", "text": "I would prefer a far-left leader with absolute authority over a right-wing leader with limited power.", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_14", "text": "Schools should be required by law to teach children about our country's history of racism, classism, sexism, and homophobia.", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_15", "text": "Anyone who opposes gay marriage must be homophobic.",                                           "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_16", "text": "Deep down, just about all conservatives are racist, sexist, and homophobic.",                     "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_17", "text": "People truly worried about terrorism should shift their focus to the nutjobs on the far-right.", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_18", "text": "The 'old-fashioned ways' and 'old-fashioned values' need to be abolished.",                        "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_19", "text": "Radical and progressive moral values can save our society.",                                     "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_20", "text": "All political conservatives are fools.",                                                         "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_21", "text": "I cannot imagine myself becoming friends with a political conservative.",                         "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_22", "text": "Conservatives are morally inferior to liberals.",                                                "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_23", "text": "It is important that we destroy the West's nationalist, imperialist values.",                     "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_24", "text": "I try to expose myself to conservative news sources.",                                           "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "LWA", "id": "LWA_25", "text": "There is nothing wrong with Bible camps.",                                                       "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "LWA", "id": "LWA_26", "text": "I hate being around nonprogressive people.",                                                     "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_27", "text": "Classroom discussions should be safe places that protect students from disturbing ideas.",        "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_28", "text": "University authorities are right to ban hateful speech from campus.",                            "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_29", "text": "I should have the right not to be exposed to offensive views.",                                  "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_30", "text": "To succeed, a workplace must ensure that its employees feel safe from criticism.",               "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_31", "text": "We must line up behind strong leaders who have the will to stamp out prejudice and intolerance.", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_32", "text": "When we spend all of our time protecting the right to “free speech” we're protecting the rights of sexists, racists, and homophobes at the cost of marginalized people", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_33", "text": "I am in favor of allowing the government to shut down right-wing internet sites and blogs that promote nutty, hateful positions", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_34", "text": "Colleges and universities that permit speakers with intolerant views should be publicly condemned.", "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_35", "text": "Getting rid of inequality is more important than protecting the so-called 'right' to free speech.",  "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_36", "text": "Fox News, right-wing talk radio, and other conservative media outlets should be prohibited from broadcasting their views.",  "scale_range": [1,7], "reverse_score": False},
    {"scale_name": "LWA", "id": "LWA_37", "text": "Even books that contain racism or racial language should not be censored.",                       "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "LWA", "id": "LWA_38", "text": "I don't support shutting down speakers with sexist, homophobic, or racist views.",               "scale_range": [1,7], "reverse_score": True},
    {"scale_name": "LWA", "id": "LWA_39", "text": "Neo-Nazis ought to have a legal right to their opinions.",                                       "scale_range": [1,7], "reverse_score": True},
]

mfq_questions = [
    {"scale_name": "MFQ", "id": "MFQ_1",  "text": "Caring for people who have suffered is an important virtue.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_2",  "text": "The world would be a better place if everyone made the same amount of money.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_3",  "text": "I think people who are more hard-working should end up with more money.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_4",  "text": "I think children should be taught to be loyal to their country.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_5",  "text": "I think it is important for societies to cherish their traditional values.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_6",  "text": "I think the human body should be treated like a temple, housing something sacred within.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_7",  "text": "I believe that compassion for those who are suffering is one of the most crucial virtues.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_8",  "text": "Our society would have fewer problems if people had the same income.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_9",  "text": "I think people should be rewarded in proportion to what they contribute.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_10", "text": "It upsets me when people have no loyalty to their country.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_11", "text": "I feel that most traditions serve a valuable function in keeping society orderly.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_12", "text": "I believe chastity is an important virtue.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_13", "text": "We should all care for people who are in emotional pain.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_14", "text": "I believe that everyone should be given the same quantity of resources in life.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_15", "text": "The effort a worker puts into a job ought to be reflected in the size of a raise they receive.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_16", "text": "Everyone should love their own community.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_17", "text": "I think obedience to parents is an important virtue.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_18", "text": "It upsets me when people use foul language like it is nothing.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_19", "text": "I am empathetic toward those people who have suffered in their lives.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_20", "text": "I believe it would be ideal if everyone in society wound up with roughly the same amount of money.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_21", "text": "It makes me happy when people are recognized on their merits.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_22", "text": "Everyone should defend their country, if called upon.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_23", "text": "We all need to learn from our elders.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_24", "text": "If I found out that an acquaintance had an unusual but harmless sexual fetish I would feel uneasy about them.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_25", "text": "Everyone should try to comfort people who are going through something hard.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_26", "text": "When people work together toward a common goal, they should share the rewards equally, even if some worked harder on it.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_27", "text": "In a fair society, those who work hard should live with higher standards of living.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_28", "text": "Everyone should feel proud when a person in their community wins in an international competition.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_29", "text": "I believe that one of the most important values to teach children is to have respect for authority.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_30", "text": "People should try to use natural medicines rather than chemically identical human-made ones.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_31", "text": "It pains me when I see someone ignoring the needs of another human being.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_32", "text": "I get upset when some people have a lot more money than others in my country.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_33", "text": "I feel good when I see cheaters get caught and punished.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_34", "text": "I believe the strength of a sports team comes from the loyalty of its members to each other.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_35", "text": "I think having a strong leader is good for society.", "scale_range": [1,5], "reverse_score": False},
    {"scale_name": "MFQ", "id": "MFQ_36", "text": "I admire people who keep their virginity until marriage.", "scale_range": [1,5], "reverse_score": False},
]

nfc_questions = [
    {"scale_name": "NFC", "id": "NFC_1", "text": "I really enjoy a task that involves coming up with new solutions to problems.", "scale_range": [-4, 4], "reverse_score": False},
    {"scale_name": "NFC", "id": "NFC_2", "text": "I believe that if I think hard enough, I will be able to achieve my goals in life.", "scale_range": [-4, 4], "reverse_score": False},
    {"scale_name": "NFC", "id": "NFC_3", "text": "I am very optimistic about my mental abilities.", "scale_range": [-4, 4], "reverse_score": False},
    {"scale_name": "NFC", "id": "NFC_4", "text": "I would prefer a task that is intellectual, difficult, and important to one that is somewhat important but does not require much thought.", "scale_range": [-4, 4], "reverse_score": False},
    {"scale_name": "NFC", "id": "NFC_5", "text": "I tend to set goals that can be accomplished only by expending considerable mental effort.", "scale_range": [-4, 4], "reverse_score": False},
    {"scale_name": "NFC", "id": "NFC_6", "text": "When something I read confuses me, I just put it down and forget it.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_7", "text": "I take pride in the products of my reasoning.", "scale_range": [-4, 4], "reverse_score": False},
    {"scale_name": "NFC", "id": "NFC_8", "text": "I don't usually think about problems that others have found to be difficult.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_9", "text": "I am usually tempted to put more thought into a task than the job minimally requires.", "scale_range": [-4, 4], "reverse_score": False},
    {"scale_name": "NFC", "id": "NFC_10", "text": "Learning new ways to think doesn't excite me very much.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_11", "text": "I am hesitant about making important decisions after thinking about them.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_12", "text": "I usually end up deliberating about issues even when they do not affect me personally.", "scale_range": [-4, 4], "reverse_score": False},
    {"scale_name": "NFC", "id": "NFC_13", "text": "I prefer just to let things happen rather than try to understand why they turned out that way.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_14", "text": "I have difficulty thinking in new and unfamiliar situations.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_15", "text": "The idea of relying on thought to make my way to the top does not appeal to me.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_16", "text": "The notion of thinking abstractly is not appealing to me.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_17", "text": "I am an intellectual.", "scale_range": [-4, 4], "reverse_score": False},
    {"scale_name": "NFC", "id": "NFC_18", "text": "I find it especially satisfying to complete an important task that required a lot of thinking and mental effort.", "scale_range": [-4, 4], "reverse_score": False},
    {"scale_name": "NFC", "id": "NFC_19", "text": "I only think as hard as I have to.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_20", "text": "I don't reason well under pressure.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_21", "text": "I like tasks that require little thought once I've learned them.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_22", "text": "I prefer to think about small, daily projects to long-term ones.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_23", "text": "I would rather do something that requires little thought than something that is sure to challenge my thinking abilities.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_24", "text": "I find little satisfaction in deliberating hard and for long hours.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_25", "text": "I think primarily because I have to.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_26", "text": "I more often talk with other people about the reasons for and possible solutions to international problems than about gossip or tidbits of what famous people are doing.", "scale_range": [-4, 4], "reverse_score": False},
    {"scale_name": "NFC", "id": "NFC_27", "text": "These days, I see little chance for performing well, even in \"intellectual\" jobs, unless one knows the right people.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_28", "text": "More often than not, more thinking just leads to more errors.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_29", "text": "I don't like to have the responsibility of handling a situation that requires a lot of thinking.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_30", "text": "I appreciate opportunities to discover the strengths and weaknesses of my own reasoning.", "scale_range": [-4, 4], "reverse_score": False},
    {"scale_name": "NFC", "id": "NFC_31", "text": "I feel relief rather than satisfaction after completing a task that required a lot of mental effort.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_32", "text": "Thinking is not my idea of fun.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_33", "text": "I try to anticipate and avoid situations where there is a likely chance I will have to think in depth about something.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_34", "text": "I don't like to be responsible for thinking of what I should be doing with my life.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_35", "text": "I prefer watching educational to entertainment programs.", "scale_range": [-4, 4], "reverse_score": False},
    {"scale_name": "NFC", "id": "NFC_36", "text": "I often succeed in solving difficult problems that I set out to solve.", "scale_range": [-4, 4], "reverse_score": False},
    {"scale_name": "NFC", "id": "NFC_37", "text": "I think best when those around me are very intelligent.", "scale_range": [-4, 4], "reverse_score": False},
    {"scale_name": "NFC", "id": "NFC_38", "text": "I am not satisfied unless I am thinking.", "scale_range": [-4, 4], "reverse_score": False},
    {"scale_name": "NFC", "id": "NFC_39", "text": "I prefer my life to be filled with puzzles that I must solve.", "scale_range": [-4, 4], "reverse_score": False},
    {"scale_name": "NFC", "id": "NFC_40", "text": "I would prefer complex to simple problems.", "scale_range": [-4, 4], "reverse_score": False},
    {"scale_name": "NFC", "id": "NFC_41", "text": "Simply knowing the answer rather than understanding the reasons for the answer to a problem is fine with me.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_42", "text": "When I am figuring out a problem, what I see as the solution to a problem is more important than what others believe or say is the solution.", "scale_range": [-4, 4], "reverse_score": False},
    {"scale_name": "NFC", "id": "NFC_43", "text": "It's enough for me that something gets the job done, I don't care how or why it works.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_44", "text": "Ignorance is bliss.", "scale_range": [-4, 4], "reverse_score": True},
    {"scale_name": "NFC", "id": "NFC_45", "text": "I enjoy thinking about an issue even when the results of my thought will have no effect on the outcome of the issue.", "scale_range": [-4, 4], "reverse_score": False}
]
# Combine questions into a single list for processing
all_scales = [mfq_questions, rwa_questions, rwa2_questions, lwa_questions, nfc_questions]  # e.g., [mfq_questions, rwa_questions, ...]

all_questions = []
for scale_list in all_scales:
    all_questions.extend(scale_list)




In [4]:

# Simple cost tracking
cost_tracker = defaultdict(int)

#Pydantic classes is best for openAi strucutred output, may be redunent when seeing main api calls below all of this, unsure but works for now. 

class ValidationError(Exception):
    pass

def validate_scale(scale_range: List[int]) -> List[int]:
    """Validate scale range is properly formatted"""
    if not (isinstance(scale_range, list) and len(scale_range) == 2 
            and isinstance(scale_range[0], int) and isinstance(scale_range[1], int)
            and scale_range[0] < scale_range[1]):
        raise ValidationError(f"Invalid scale range: {scale_range}")
    return scale_range


class SurveyAnswer(BaseModel):
    numeric_score: float
    label: str = ""  # Make optional with default
    justification: str = ""  

def safe_parse_survey_answer(response_text: str, scale_range: List[int]) -> Optional[SurveyAnswer]:
    """
    Simplified parser that handles both structured and unstructured responses.
    Prioritizes finding valid numeric scores within the scale range.
    """
    response_text = str(response_text).strip().lower()  # Normalize text
    min_scale, max_scale = scale_range
    
    def is_valid_score(num: float) -> bool:
        return min_scale <= num <= max_scale

    try:
        # 1. Simple format: "Rating: X" or "Score: X"
        simple_match = re.search(r'(?:rating|score):\s*(-?\d+(?:\.\d+)?)', response_text)
        if simple_match:
            num = float(simple_match.group(1))
            if is_valid_score(num):
                # Get everything after the rating as justification
                explanation = response_text[simple_match.end():].strip()
                return SurveyAnswer(
                    numeric_score=num,
                    justification=explanation or response_text
                )

        # 2. Try JSON parsing (for OpenAI, Claude, Grok)
        try:
            data = json.loads(response_text)
            if 'rating' in data and is_valid_score(float(data['rating'])):
                return SurveyAnswer(
                    numeric_score=float(data['rating']),
                    justification=data.get('justification', '')
                )
        except json.JSONDecodeError:
            pass

        # 3. Find first valid number in text
        numbers = re.findall(r'-?\d+(?:\.\d+)?', response_text)
        for num_str in numbers:
            try:
                num = float(num_str)
                if is_valid_score(num):
                    return SurveyAnswer(
                        numeric_score=num,
                        justification=response_text
                    )
            except ValueError:
                continue

        # 4. Fallback: use scale midpoint
        midpoint = (min_scale + max_scale) / 2
        logger.warning(f"No valid number found in: {response_text[:100]}...")
        return SurveyAnswer(
            numeric_score=midpoint,
            justification=f"PARSER WARNING: No valid number found in response"
        )

    except Exception as e:
        logger.error(f"Parser error: {str(e)}")
        midpoint = (min_scale + max_scale) / 2
        return SurveyAnswer(
            numeric_score=midpoint,
            justification=f"PARSER ERROR: {str(e)}"
        )

# call_model_api function to pass scale_range
async def call_model_api(model_name, question_text, prompt_style_key, scale_range, temperature=0.0) -> Dict:
    start_time = time.time()
    parsed, raw_response = await call_model(model_name, question_text, prompt_style_key, scale_range, temperature)
    
    # If parsing failed, create a fallback SurveyAnswer with scale midpoint
    if parsed is None:
        min_scale, max_scale = scale_range
        midpoint = (min_scale + max_scale) / 2
        parsed = SurveyAnswer(
            numeric_score=midpoint,
            justification=f"API ERROR: {raw_response[:100]}..."
        )
    
    elapsed = time.time() - start_time
    return {
        "model_name": model_name,
        "numeric_score": parsed.numeric_score,  # Will always have a value 
        "label": parsed.label,
        "justification": parsed.justification,
        "raw_response": raw_response,
        "duration": elapsed
    }

In [5]:
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
#again another async definiton on top of others, First time using async my self so still probally being reduntant. 
async def call_model(
    model_name: str,
    question_text: str,
    prompt_style_key: str,
    scale_range: List[int],
    temperature: float = 0.0
) -> Tuple[Optional[SurveyAnswer], str]:
    """
    Creates the final prompt and calls the appropriate client.
    """
    validate_scale(scale_range)
    
    config = MODEL_CONFIG[model_name]
    style_prompt = prompt_templates[prompt_style_key]
    scale_str = f"(Scale from {scale_range[0]} to {scale_range[1]})"

    final_prompt = f"""{style_prompt}

Question: {question_text}
{scale_str}

Please provide your response in JSON format:
{{"rating": <number>, "justification": "<explanation>"}}"""

    raw_text = ""
    try:
        # ------------------------------------------------------------------
        # 1) OpenAI => AsyncOpenAI usage
        # ------------------------------------------------------------------
        if model_name == "OpenAI":
            client = AsyncOpenAI(api_key=config["api_key"])
            response = await client.chat.completions.create(
                model=config["model"],
                messages=[{"role": "user", "content": final_prompt}],
                temperature=temperature,
                max_tokens=512,
            )
            if hasattr(response, 'usage'):
                cost_tracker[model_name] += (
                    response.usage.prompt_tokens + response.usage.completion_tokens
                )
            raw_text = response.choices[0].message.content
            parsed = safe_parse_survey_answer(raw_text, scale_range)
            return parsed, raw_text

        # ------------------------------------------------------------------
        # 2) Grok => same AsyncOpenAI approach but with base_url
            #it may make sense to combine OpenAi call and grok ased on nearly exaxt similar strucutre. 
        # ------------------------------------------------------------------
        elif model_name == "Grok":
            base_url = config["base_url"]
            client = AsyncOpenAI(api_key=config["api_key"], base_url=base_url)
            response = await client.chat.completions.create(
                model=config["model"],
                messages=[{"role": "user", "content": final_prompt}],
                temperature=temperature,
                max_tokens=512,
            )
            if hasattr(response, 'usage'):
                cost_tracker[model_name] += (
                    response.usage.prompt_tokens + response.usage.completion_tokens
                )
            raw_text = response.choices[0].message.content
            parsed = safe_parse_survey_answer(raw_text, scale_range)
            return parsed, raw_text
        # ------------------------------------------------------------------
        # 3) Claude => Using AsyncAnthropic client
        #claude was acting weird with strucutred output, probally due to my over zealous use of my parser or the fact I am winging most of this. 
        # ------------------------------------------------------------------
        elif model_name == "Claude":
            client = AsyncAnthropic(api_key=config["api_key"])
            
            response = await client.messages.create(
                model=config["model"],
                max_tokens=1024,
                messages=[
                    {
                        "role": "user",
                        "content": final_prompt
                    }
                ],
            )
            
            # Get content from response
            raw_text = ""
            if hasattr(response, 'content'):
                for content_block in response.content:
                    if hasattr(content_block, 'text'):
                        raw_text += content_block.text
            
            # Track tokens if available
            if hasattr(response, 'usage'):
                cost_tracker[model_name] += (
                    response.usage.input_tokens + response.usage.output_tokens
                )
            
            parsed = safe_parse_survey_answer(raw_text, scale_range)
            return parsed, raw_text
        # ------------------------------------------------------------------
        # 4) Llama or DeepSeek 
        #having trouble with AsyncCalls using "Llama Stack"(apperntly that is the name of the package that can handle some async calls with various open source models)
        #using basic sequential calls now which means I never call this function, need help..
        #also having isssues with consitent output for parser, again my fault for not having a better parser pipline.
        # ------------------------------------------------------------------
    
        elif model_name in ["Llama", "DeepSeek"]:
            llama_client = LlamaAPI(config["api_key"])
            
            request_data = {
                "model": config["model"],
                "messages": [
                    {"role": "user", "content": final_prompt}
                ],
                "stream": False
            }
            
            def run_llama():
                try:
                    response = llama_client.run(request_data)
                    raw_text = response.json()["choices"][0]["message"]["content"]
                    
                    # Track tokens if available
                    if "usage" in response.json():
                        cost_tracker[model_name] += response.json()["usage"].get("total_tokens", 0)
                    
                    return safe_parse_survey_answer(raw_text, scale_range), raw_text
                    
                except Exception as e:
                    logger.error(f"Llama API error: {str(e)}")
                    return None, str(e)
            
            loop = asyncio.get_running_loop()
            parsed, raw_text = await loop.run_in_executor(None, run_llama)
            return parsed, raw_text


        else:
            return None, f"Error: Unknown model {model_name}"

    except Exception as e:
        logger.error(f"Error calling {model_name}: {str(e)}")
        raise  # Let retry handle it

In [6]:
# -------------------------------------------------------------------------
# A small wrapper to measure time & return a dict of info
#this may be a bit extra but could be useful for calculating energy and c02 costs. 
# -------------------------------------------------------------------------
async def call_model_api(model_name, question_text, prompt_style_key, scale_range, temperature=0.0) -> Dict:
    start_time = time.time()
    parsed, raw_response = await call_model(model_name, question_text, prompt_style_key, scale_range, temperature)
    
    # If parsing failed, created a fallback SurveyAnswer with scale midpoint, this could cuase varience in the future, but minimal as of now.
    if parsed is None:
        min_scale, max_scale = scale_range
        midpoint = (min_scale + max_scale) / 2
        parsed = SurveyAnswer(
            numeric_score=midpoint,
            justification=f"API ERROR: {raw_response[:100]}..."
        )
    
    elapsed = time.time() - start_time
    return {
        "model_name": model_name,
        "numeric_score": parsed.numeric_score,  # Will always have a value 
        "label": parsed.label,
        "justification": parsed.justification,
        "raw_response": raw_response,
        "duration": elapsed
    }

In [12]:
# --------------------------------------------------------------------------------------
# Global Parameters
# --------------------------------------------------------------------------------------
SCALES_TO_RUN = ["LWA", "RWA2"]  # e.g., ["NFC", "MFQ", "RWA", "LWA"]
PROMPT_STYLES_TO_RUN = ["neutral", "extreme_liberal", "extreme_conservitive"] # ["neutral", "mid_conservitive", "mid_liberal",  "extreme_liberal", "extreme_conservitive"]
NUM_CALLS_TEST = 20
MODELS_TO_RUN = ["OpenAI", "Claude", "Grok"] # ["OpenAI", "Claude", "Llama", "Grok", "DeepSeek"]
TEMPERATURE = 0
MAX_CONCURRENT_CALLS = 5

# --------------------------------------------------------------------------------------
# Build the Tasks
# --------------------------------------------------------------------------------------
tasks = []
for q in all_questions:
    if q["scale_name"] not in SCALES_TO_RUN:
        continue
    scale_name = q["scale_name"]
    question_id = q["id"]
    question_text = q["text"]
    scale_range = q["scale_range"]
    reverse_score = q.get("reverse_score", False)

    for model_name in MODELS_TO_RUN:
        for prompt_style in PROMPT_STYLES_TO_RUN:
            for run in range(1, NUM_CALLS_TEST + 1):
                tasks.append({
                    "model_name": model_name,
                    "scale_name": scale_name,
                    "question_id": question_id,
                    "question_text": question_text,
                    "prompt_style": prompt_style,
                    "run_number": run,
                    "scale_range": scale_range,
                    "reverse_score": reverse_score,
                })

# (Optionally limit tasks for quick testing)
#tasks = tasks[:20]


In [13]:
from tqdm.notebook import tqdm  # Specifically for Jupyter notebooks

async def process_tasks_in_chunks(task_list, chunk_size=5):
    """Process tasks with improved concurrency and rate limiting"""
    results = []
    total_tasks = len(task_list)
    
    # Create separate queues for different API providers to avoid rate limits
    openai_queue = []
    anthropic_queue = []
    llama_queue = []
    
    # Sort tasks by API provider
    for task in task_list:
        if task["model_name"] in ["OpenAI", "Grok"]:
            openai_queue.append(task)
        elif task["model_name"] == "Claude":
            anthropic_queue.append(task)
        else:  # Llama and DeepSeek
            llama_queue.append(task)
    
    # Create progress bar
    pbar = tqdm(total=total_tasks, desc="Processing tasks")
    
    async def process_queue(queue, semaphore, rate_limit):
        """Process a queue with rate limiting"""
        queue_results = []
        for i in range(0, len(queue), chunk_size):
            chunk = queue[i:i + chunk_size]
            async with semaphore:
                # Process chunk with rate limiting
                coros = [
                    call_model_api(
                        t["model_name"],
                        t["question_text"],
                        t["prompt_style"],
                        t["scale_range"],
                        0.0
                    )
                    for t in chunk
                ]
                chunk_results = await asyncio.gather(*coros, return_exceptions=True)
                queue_results.extend(zip(chunk, chunk_results))
                
                # Update progress bar
                pbar.update(len(chunk))
                
                # Rate limiting delay
                await asyncio.sleep(rate_limit)
        
        return queue_results

    # Create separate semaphores for each API provider
    openai_sem = asyncio.Semaphore(3)    # Allow 3 concurrent OpenAI calls
    anthropic_sem = asyncio.Semaphore(5)  # Allow 5 concurrent Anthropic calls
    llama_sem = asyncio.Semaphore(10)     # Allow 10 concurrent Llama calls
    
    # Process all queues concurrently with different rate limits
    queue_tasks = [
        process_queue(openai_queue, openai_sem, 1.0),      # 1 second between chunks
        process_queue(anthropic_queue, anthropic_sem, 0.5), # 0.5 seconds between chunks
        process_queue(llama_queue, llama_sem, 0.2),        # 0.2 seconds between chunks
    ]
    
    all_results = await asyncio.gather(*queue_tasks)
    
    # Close progress bar
    pbar.close()
    
    # Combine and process results
    for queue_result in all_results:
        for task_info, result in queue_result:
            if isinstance(result, Exception):
                logger.error(f"Task failed: {str(result)}")
                # Use scale midpoint for failed tasks
                min_scale, max_scale = task_info["scale_range"]
                midpoint = (min_scale + max_scale) / 2
                result_dict = {
                    "model_name": task_info["model_name"],
                    "numeric_score": midpoint,
                    "label": None,
                    "justification": f"ERROR: {str(result)}",
                    "raw_response": str(result),
                    "duration": None
                }
            else:
                result_dict = result
            
            # Validate numeric score
            min_scale, max_scale = task_info["scale_range"]
            if not (min_scale <= result_dict["numeric_score"] <= max_scale):
                logger.warning(f"Score {result_dict['numeric_score']} out of range for {task_info['question_id']}, using midpoint")
                result_dict["numeric_score"] = (min_scale + max_scale) / 2
                result_dict["justification"] = f"RANGE ERROR: Original score: {result_dict['numeric_score']}"
            
            result_dict.update(task_info)
            results.append(result_dict)
    
    logger.info(f"Processed {len(results)}/{total_tasks} tasks. Current token usage: {dict(cost_tracker)}")
    
    return results

In [14]:

# -------------------------------------------------------------------------
# Run everything, build DataFrame, save CSV
# -------------------------------------------------------------------------
results = asyncio.run(process_tasks_in_chunks(tasks))

def apply_reverse_score(row):
    """
    Apply reverse scoring for different scales.
    - MFQ scale is 1-5; reversed item => 6 - original
    - LWA and RWA scale is 1-7; reversed item => 8 - original
    """
    score = row["numeric_score"]
    reverse_flag = row.get("reverse_score", False) #may be the wrong way to iterate this, but it works for now.
    scale_name = row.get("scale_name", "")

    if pd.isna(score):
        return score  # No change if score is NaN

    if not reverse_flag:
        return score  # Return as-is if not a reverse-scored item

    # Handle each scale's reversing logic
    if scale_name == "MFQ":
        return 6 - score
    elif scale_name == "RWA":
        return 8 - score  # Same logic as LWA for 1-7 scale
    elif scale_name == "RWA2":
        return 8 - score  # Same logic as LWA for 1-7 scale
    elif scale_name == "LWA":
        return 8 - score
    else:
        # If any future scale needs reversing, define it here
        return score

# Build a DataFrame of the raw results
df_results = pd.DataFrame(results)

# Apply reverse scoring
df_results["scored_value"] = df_results.apply(apply_reverse_score, axis=1)

# Save to CSV (unified responses without any final summations)
df_results.to_csv("unified_responses.csv", index=False)
print("Saved responses to unified_responses.csv")

# Print summary info
print("\nProcessing Summary:")
print(f"Total responses: {len(df_results)}")
print(f"Token usage by model: {dict(cost_tracker)}")
resp_rate = df_results['numeric_score'].notna().sum() / len(df_results) * 100
print(f"Response rate: {resp_rate:.1f}%")

Processing tasks:   0%|          | 0/10980 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request:

Saved responses to unified_responses.csv

Processing Summary:
Total responses: 10980
Token usage by model: {'OpenAI': 1475429, 'Claude': 1642969, 'Grok': 1324983}
Response rate: 100.0%


In [ ]:
def save_refusal_responses(df, output_file="refusal_responses.csv"):
    """
    After generating unified_responses.csv, call this function to save 
    any rows where the model refused or failed to provide a valid numeric answer.

    Specifically, we look for:
    - The 'justification' that indicates a parser warning or API error.
    - The 'raw_response' text where we logged: 'No valid number found in: ...'

    Adjust filters as needed for your exact logging conventions.
    """
    # Filter rows based on how your warnings/errors are recorded
    df_refusals = df[
        df["justification"].str.contains("PARSER WARNING", na=False)
        | df["justification"].str.contains("API ERROR", na=False)
        | df["raw_response"].str.contains("No valid number found in:", na=False)
    ].copy()

    # Choose the columns that best help you analyze the refusal
    columns_to_save = [
        "model_name",
        "prompt_style",
        "question_id",
        "question_text",
        "justification",
        "raw_response"
    ]
    # Only keep columns actually present
    columns_to_save = [col for col in columns_to_save if col in df_refusals.columns]

    df_refusals = df_refusals[columns_to_save]
    df_refusals.to_csv(output_file, index=False)
    print(f"Refusal responses saved to {output_file}")



df_results = pd.read_csv("unified_responses.csv")
save_refusal_responses(df_results, output_file="refusal_responses.csv")


In [ ]:
# Define MFQ question groupings for each foundation
MFQ_FOUNDATIONS = {
    'care': [f'MFQ_{i}' for i in [1, 7, 13, 19, 25, 31]],
    'equality': [f'MFQ_{i}' for i in [2, 8, 14, 20, 26, 32]],
    'proportionality': [f'MFQ_{i}' for i in [3, 9, 15, 21, 27, 33]],
    'loyalty': [f'MFQ_{i}' for i in [4, 10, 16, 22, 28, 34]],
    'authority': [f'MFQ_{i}' for i in [5, 11, 17, 23, 29, 35]],
    'purity': [f'MFQ_{i}' for i in [6, 12, 18, 24, 30, 36]]
}

def calculate_mfq_scores(csv_path):
    # Read CSV
    df = pd.read_csv(csv_path)
    
    # Filter for MFQ questions only
    df = df[df['scale_name'] == 'MFQ']
    
    # First, average scores across runs for each unique combination
    # of model, prompt, and question....this may be a bad way to do this? perhaps not the mean but the medium instead? 
    avg_by_question = df.groupby([
        'model_name',
        'prompt_style',
        'question_id'
    ])['numeric_score'].mean().reset_index()
    
    results = []
    
    # Process each model/prompt combination
    for model in avg_by_question['model_name'].unique():
        for prompt_style in avg_by_question['prompt_style'].unique():
            row = {'model_name': model, 'prompt_style': prompt_style}
            
            # Calculate score for each foundation
            for foundation, questions in MFQ_FOUNDATIONS.items():
                mask = (avg_by_question['model_name'] == model) & \
                    (avg_by_question['prompt_style'] == prompt_style) & \
                    (avg_by_question['question_id'].isin(questions))
                
                foundation_scores = avg_by_question[mask]['numeric_score']
                
                if len(foundation_scores) > 0:
                    # Calculate foundation metrics
                    row[f'{foundation}_mean'] = round(foundation_scores.mean(), 2)
                    row[f'{foundation}_std'] = round(foundation_scores.std(), 2)
                    row[f'{foundation}_count'] = len(foundation_scores)
                    # Add individual question scores for verification
                    for q_id in questions:
                        q_score = avg_by_question[
                            (avg_by_question['model_name'] == model) & 
                            (avg_by_question['prompt_style'] == prompt_style) & 
                            (avg_by_question['question_id'] == q_id)
                        ]['numeric_score'].values
                        if len(q_score) > 0:
                            row[f'{q_id}_score'] = round(q_score[0], 2)
                        else:
                            row[f'{q_id}_score'] = None
                else:
                    row[f'{foundation}_mean'] = None
                    row[f'{foundation}_std'] = None
                    row[f'{foundation}_count'] = 0
                    for q_id in questions:
                        row[f'{q_id}_score'] = None
            
            results.append(row)
    
    # Convert to DataFrame and sort by model name and prompt style
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(['model_name', 'prompt_style'])
    
    # Save results
    results_df.to_csv('mfq_foundation_scores.csv', index=False)
    print("\nMFQ Foundation Scores:")
    print(results_df.to_string())
    
    return results_df

# Run it
calculate_mfq_scores('unified_responses.csv')